<a href="https://colab.research.google.com/github/OwenBellGit/CP468_Project_GoldenEagle/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow keras gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 867.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import cv2
import gradio as gr

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Defining paths of test and training images
train_dataset_path = '/content/drive/MyDrive/CP468_Proj_Data/train'
test_dataset_path = '/content/drive/MyDrive/CP468_Proj_Data/test2'
valid_dataset_path = '/content/drive/MyDrive/CP468_Proj_Data/valid'

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # Randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.15,  # Randomly zoom image
    shear_range=0.15,  # Shear intensity (shear angle in counter-clockwise direction in degrees)
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    classes=['GoldenEagle', 'NotGoldenEagle'],
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    valid_dataset_path,
    classes=['GoldenEagle', 'NotGoldenEagle'],
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    classes=['GoldenEagle', 'NotGoldenEagle'],
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

Found 1226 images belonging to 2 classes.
Found 110 images belonging to 2 classes.
Found 282 images belonging to 2 classes.


In [12]:
# Load the InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the Base Layer of the model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers for classification
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Earlystopping callback to stop after the model stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model
model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Save the trained model
model.save('/content/drive/MyDrive/fine_tuned_inception_v3.keras')

Epoch 1/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.7230 - loss: 0.7956 - val_accuracy: 0.9000 - val_loss: 0.2630
Epoch 2/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.8693 - loss: 0.3422 - val_accuracy: 0.8909 - val_loss: 0.2977
Epoch 3/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.8750 - loss: 0.3057 - val_accuracy: 0.9091 - val_loss: 0.2380
Epoch 4/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.8682 - loss: 0.3351 - val_accuracy: 0.7818 - val_loss: 0.4733
Epoch 5/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.9227 - loss: 0.2274 - val_accuracy: 0.9182 - val_loss: 0.2234
Epoch 6/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.8939 - loss: 0.2775 - val_accuracy: 0.8727 - val_loss: 0.3339
Epoch 7/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9136 - loss: 0.2163 - val_accuracy: 0.9000 - val_loss: 0.2233
Epoch 8/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9262 - loss: 0.1785 - val_accuracy: 0.8636 - val_loss:

In [15]:
# Reset the test generator and make predictions
test_generator.reset()
predictions = model.predict(test_generator)
pred = (predictions > 0.5).astype(int).flatten()

# Get the labels
actual = test_generator.classes
labels = list(test_generator.class_indices.keys())

# Calculate the Confusion Matrix
cnf_mtx = tf.math.confusion_matrix(actual, pred)
print(cnf_mtx)

9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 955ms/step
tf.Tensor(
[[  4  16]
 [ 24 238]], shape=(2, 2), dtype=int32)


In [16]:
print("Classification Report")
print(classification_report(actual, pred, target_names=labels))

Classification Report
                precision    recall  f1-score   support

   GoldenEagle       0.14      0.20      0.17        20
NotGoldenEagle       0.94      0.91      0.92       262

      accuracy                           0.86       282
     macro avg       0.54      0.55      0.54       282
  weighted avg       0.88      0.86      0.87       282



In [18]:
labels = ['GoldenEagle', 'NotGoldenEagle']

def classify_image(img):
    # Preprocess
    img = cv2.resize(img, (128, 128))
    img = img.reshape((-1, 128, 128, 3))
    img = tf.keras.applications.inception_v3.preprocess_input(img)


    # Make prediction
    prediction = model.predict(img).flatten()
    probability = float(prediction[0])
    return {
        labels[0]: probability,
        labels[1]: 1.0 - probability
    }

# Define Gradio interface
image = gr.components.Image()
label = gr.components.Label(num_top_classes=2)


# Create the Gradio interface
iface = gr.Interface(
    fn=classify_image,  # Function to be executed
    inputs=image,       # Input component (image upload)
    outputs=label,      # Output component (displays labels)
)

# Launch the Gradio interface to allow users to interact with the model
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://47184f7e6a567f86ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://47184f7e6a567f86ef.gradio.live
